# Deep Dive into LangChain

In [1]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


### Python-dotenv

In [2]:
import os 
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

'sk-proj-dsxvcH7KXpdycO17nObyT3BlbkFJivJ8OhSW43DNJlVp6w1D'

## Chat Models: GPT-3.5 Turbo and GPT-4

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
# output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', max_tokens=100, temperature=0.5, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=['\n', ')
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo')
print(output.content)

Quantum mechanics is the branch of physics that describes the behavior of particles on a very small scale, where the principles of classical physics no longer apply and instead, phenomena are governed by probabilities and uncertainties.


In [4]:
from langchain.schema import (
    SystemMessage, # corresponds to the OpenAI chat completion API
    AIMessage,     # Assistent message
    HumanMessage   # Human messages or prompt
)

messages = [    
    SystemMessage(content='You are a physicist and respond only in portuguese.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

A mecânica quântica é a teoria que descreve o comportamento das partículas subatômicas através de conceitos como superposição, interferência e entrelaçamento quântico.


## Caching LLM responses

### In-Memory cache

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [6]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke about quantum mechanics.'
llm.invoke(prompt)

CPU times: user 2.19 s, sys: 446 ms, total: 2.63 s
Wall time: 3.13 s


'\n\nWhy was the quantum physicist always uncertain?\n\nBecause he could never be sure if he was a particle or a wave!'

After storing it in cache, the time to run the cell decreases to zero!!!

In [7]:
%%time
llm.invoke(prompt)

CPU times: user 458 μs, sys: 74 μs, total: 532 μs
Wall time: 539 μs


'\n\nWhy was the quantum physicist always uncertain?\n\nBecause he could never be sure if he was a particle or a wave!'

### SQLite Caching

(to store in the SQLite caching)

In [8]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

prompt = 'Tell me a joke about quantum mechanics.'

# First request (not in cache, takes longer)
llm.invoke(prompt)

# Second request (in cache, takes less time)
llm.invoke(prompt)

"\n\nWhy did Schrödinger's cat refuse to come out of the box?\n\nBecause it was afraid of collapsing the waveform!"